### Data Cleaning

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
from io import open

# Load the file
file_path = '7e97a6db-e083-410c-b84d-05240030faa9__2025_03_22T19_19_40.tsv.xz'
df = pd.read_csv(file_path, sep='\t', compression='xz')

# Clean column names (remove extra spaces)
df.columns = df.columns.str.strip()

# Function to clean text by removing HTML tags, URLs, and extra spaces
def clean_text(text):
    if pd.isna(text):  # Return empty string if value is NaN
        return ''
    text = re.sub(r'<[^>]+>', ' ', text)  # Remove HTML tags
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)  # Remove URLs
    text = re.sub(r'&[a-zA-Z0-9#]+;', ' ', text)  # Remove HTML entities
    text = re.sub(r'\s+', ' ', text).strip()  # Reduce multiple spaces
    return text

def remove_numbered_links(df):
    # Create a dictionary to store the base links and their corresponding rows
    link_dict = {}
    nan_rows = []

    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        # If the article_link is NaN, add the row index to nan_rows
        if pd.isna(row['article_link']):
            nan_rows.append(index)
            continue

        # Extract the base link by removing the trailing '-number' if present at the end
        base_link = re.sub(r'-\d+$', '', row['article_link'])

        # If the base link is not in the dictionary, add it
        if base_link not in link_dict:
            link_dict[base_link] = index
        else:
            # If the base link is already in the dictionary, compare the current row with the stored row
            existing_index = link_dict[base_link]
            if re.search(r'-\d+$', df.at[existing_index, 'article_link']) and df.at[existing_index, 'head'] == row['head']:
                # If the existing link has a '-number' suffix and the titles are the same, replace it with the current row
                link_dict[base_link] = index

    # Get the indices to keep, including rows with NaN article_link
    indices_to_keep = list(link_dict.values()) + nan_rows

    # Filter the DataFrame to keep only the rows with the desired indices
    df = df.loc[indices_to_keep].reset_index(drop=True)

    return df

def remove_similar_rows(df, threshold=0.995):
    # Drop rows where the content_id is the same
    df = df.drop_duplicates(subset=['content_id'])

    # Clean the content column
    df.loc[:, 'content'] = df['content'].apply(clean_text)

    # Drop rows where the content is exactly the same
    df = df.drop_duplicates(subset=['content'])

    df = remove_numbered_links(df)

    # Vectorize the content using TF-IDF
    vectorizer = TfidfVectorizer().fit_transform(df['content'])
    vectors = vectorizer.toarray()

    # Compute cosine similarity matrix
    cosine_sim_matrix = cosine_similarity(vectors)

    # Identify pairs of articles with similarity above the threshold
    similar_pairs = np.where(cosine_sim_matrix > threshold)

    # Create a set of indices to drop
    indices_to_drop = set()
    for i, j in zip(*similar_pairs):
        if i != j:
            indices_to_drop.add(j)

    # Drop the duplicates using .loc to avoid SettingWithCopyWarning
    df = df.loc[~df.index.isin(indices_to_drop)]

    # Reset index
    df.reset_index(drop=True, inplace=True)

    return df

# Drop same or nearly same articles
df = remove_similar_rows(df, 0.65)

with open('output_file.tsv', 'w') as f:
    df.to_csv(f, sep='\t', index=False)

# make the pubtime a df datetime format
df['pubtime'] = pd.to_datetime(df['pubtime'])

df

,id,pubtime,medium_code,medium_name,char_count,head,article_link,content_id,content
0,56411905,2025-03-20 10:25:38+01:00,NNTA,tagesanzeiger.ch,2406,Flughafen Zürich: Swiss-Flug nach Budapest mus...,https://www.tagesanzeiger.ch/flughafen-zuerich...,0031313b-f8f4-5281-70d8-9727db8ef8f6,"Ein Flug der Swiss, durchgeführt von AirBaltic..."
1,56360926,2025-03-14 17:44:58+01:00,NNBE,bernerzeitung.ch,1331,Zu hohe Temperaturen: Rennabbruch an der Bob-WM,https://www.bernerzeitung.ch/bob-wm-lake-placi...,003aa8b4-f90d-63f2-636c-33a3b91f0aea,Die Sonne sorgt an der Bob-WM für schwierige V...
2,56349050,2025-03-13 14:53:13+01:00,SRF,srf.ch,2595,Mit den «Drachen» zum selben Höhenflug ansetze...,https://www.srf.ch/sport/eishockey/national-le...,007555d5-6e16-cb17-b0d0-213bc53cb964,2016 wurde Leuenberger mit dem SC Bern als Int...
3,56370481,2025-03-15 23:35:40+01:00,NNBE,bernerzeitung.ch,1861,Eishockey-Playoffs: Lausanne muss den Ausgleic...,https://www.bernerzeitung.ch/eishockey-tigers-...,00a0ad13-aae5-160e-e51c-f7ddaa40841e,Lausanne verliert im zweiten Playoff-Viertelfi...
4,56372428,2025-03-16 09:41:18+01:00,SRF,srf.ch,3064,Norris' «stressiger» Sieg – Hülkenberg mit «Ba...,https://www.srf.ch/sport/motorsport/formel-1/s...,00e2d2b8-b04f-1963-5af5-60e5eeddf84f,Erleichterung bei Sauber nach dem GP Australie...
...,...,...,...,...,...,...,...,...,...
4443,56340876,2025-03-13 00:00:00+01:00,NZZ,Neue Zürcher Zeitung,12728,Die Labor-These gewinnt wieder an Gewicht,NaN,ffbb940a-c766-37ed-6f9b-bded485bf53b,"Johannes Boie, Berlin Der deutschen Bundesregi..."
4444,56329575,2025-03-12 00:00:00+01:00,BLI,Blick,2737,Lemieux – der Mann fürs Grobe beim HCD,NaN,ffcd9173-60dd-68ce-6603-36f63f76c0ca,"Brendan Lemieux (28) macht keinen Hehl daraus,..."
4445,56345547,2025-03-13 00:00:00+01:00,WEW,Die Weltwoche,5286,Friedrich Merz und der Schulden-Tsunami,NaN,ffce0fee-1078-691a-2e49-dcbf33efb38e,Prof. Stefan Homburg Hannover Nach dem Zerbrec...
4446,56395580,2025-03-19 00:00:00+01:00,OLT,Oltner Tagblatt,3172,«Wollen machen statt jammern»,NaN,ffd09e01-c429-b69f-cc68-2f68952469bf,Bei den Trimbacher Gemeinderatswahlen tritt mi...


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
import plotly.express as px

def df_plot_dbscan(df):
    # Combine the header and the cleaned content
    df = df.copy()  # Create a copy to avoid SettingWithCopyWarning
    df.loc[:, "combined_text"] = df["head"] + " " + df["content"]

    # Load a high-performance German-specific Sentence Transformer
    model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')

    # Convert texts to embeddings
    embeddings = model.encode(df['combined_text'].tolist())

    # Initialize DBSCAN parameters
    eps = 0.80
    min_samples = 7
    target_clusters = 4
    max_clusters = 5
    max_iterations = 100
    iteration = 0

    while iteration < max_iterations:
        # DBSCAN Clustering with the current hyperparameters
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        df.loc[:, 'dbscan_cluster'] = dbscan.fit_predict(embeddings)

        # Count the number of clusters (excluding noise points labeled as -1)
        num_clusters = len(set(df['dbscan_cluster'])) - (1 if -1 in df['dbscan_cluster'] else 0)

        if target_clusters <= num_clusters <= max_clusters:
            break

        # Adjust the eps value
        if num_clusters > target_clusters:
            eps = max(eps - 0.01, 0.01)  # Ensure eps does not go below 0.01
        else:
            eps += 0.01

        iteration += 1

    # Apply t-SNE for dimensionality reduction to 2 dimensions
    tsne = TSNE(n_components=2, random_state=42)
    tsne_results = tsne.fit_transform(embeddings)

    # Add the t-SNE results and clusters to the DataFrame
    df.loc[:, 'tsne_x'] = tsne_results[:, 0]
    df.loc[:, 'tsne_y'] = tsne_results[:, 1]

    # Create an interactive Plotly visualization for DBSCAN with the current parameters
    fig_dbscan = px.scatter(
        df[df["dbscan_cluster"] >= 0],
        x='tsne_x',
        y='tsne_y',
        color='dbscan_cluster',
        hover_data=['head'],
        title=f't-SNE Visualization of Text Clusters (DBSCAN: eps={dbscan.eps}, min_samples={dbscan.min_samples})'
    )

    for cluster_id in range(len(df['dbscan_cluster'].unique()) - 1):
        cluster_data = df[df['dbscan_cluster'] == cluster_id]
        print(f"Cluster {cluster_id} heads:")
        for title in cluster_data['head'].tolist():
            print("   " + title)

    # Show the DBSCAN plot
    fig_dbscan.show()

# Get unique dates from the pubtime column
unique_dates = df['pubtime'].dt.date.unique()

for date in unique_dates:
    subset_df = df[df['pubtime'].dt.date == date]
    print(f"Topics of {date}:")
    df_plot_dbscan(subset_df)

C:\Users\sfl\Desktop\Do-it_Einzugsgebiete\.venv\WAVE\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Topics of 2025-03-20:


No sentence-transformers model found with name T-Systems-onsite/cross-en-de-roberta-sentence-transformer. Creating a new one with mean pooling.
C:\Users\sfl\Desktop\Do-it_Einzugsgebiete\.venv\WAVE\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sfl\.cache\huggingface\hub\models--T-Systems-onsite--cross-en-de-roberta-sentence-transformer. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/w

KeyboardInterrupt: 